# Import dependencies

In [1]:
import cleverhans
from cleverhans.attacks import ProjectedGradientDescent
from cleverhans.attacks import FastGradientMethod
from cleverhans.model import Model as CleverhansModel

from io import BytesIO

import lucid.modelzoo.vision_models as models
import lucid.optvis.render as render
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import requests
import tensorflow as tf
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input

import os

Using TensorFlow backend.


In [2]:
tf.logging.set_verbosity(tf.logging.ERROR)

# Define model wrapper for cleverhans

In [3]:
class InceptionV1Model(CleverhansModel):
    SCOPE = 'inceptionv1'
    SOFTMAX_OP = 'softmax2'
  
    def __init__(self):
        super(InceptionV1Model, self).__init__()
        self._model = models.InceptionV1()
        self._model.load_graphdef()
        self._fprop_cache = dict()

    def fprop(self, x):
        if x not in self._fprop_cache:
            # https://github.com/tensorflow/lucid/blob/67e19f38c315e548034c3e4315dfee6f718df916/lucid/modelzoo/vision_base.py#L189
            graph = x.graph
            scope = graph.unique_name(
              '%sfprop%d' % (self.SCOPE, len(self._fprop_cache)), False)
            _, x_prep = self._model.create_input(x, forget_xy_shape=True)
            final_input_map = {self._model.input_name: x_prep}
            ops = tf.import_graph_def(self._model.graph_def, final_input_map,
                                    return_elements=[self.SOFTMAX_OP], name=scope)
            self._model.post_import(scope)

            softmax2_op = ops[0]
            logits = softmax2_op.inputs[0]
            logits = logits[:, :1000]

            self._fprop_cache[x] = {'logits' : logits}

        return self._fprop_cache[x]

# Define utility functions

In [4]:
def load_image_from_url(img_url, resize_to=(224, 224)):
    img_response = requests.get(img_url)
    img = Image.open(BytesIO(img_response.content))
    if resize_to is not None:
        img = img.resize(resize_to)
    img = np.array(img)
    return img

In [5]:
def load_image_from_file(filepath):
    img = image.load_img(filepath, target_size=(224, 224))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    return img

In [6]:
def get_predictions(model, imgs):
    graph = tf.Graph()
    with tf.Session(graph=graph) as sess:  
        x = tf.placeholder(tf.float32, (None, 224, 224, 3))
        y = model.get_probs(x)
        y = tf.arg_max(y, 1)
        y_eval = sess.run(y, feed_dict={x: imgs})

    return y_eval

In [7]:
def attack_images(model, imgs, attack_type="FGM", attack_kwargs={'eps': 0.3, 'y_target': 0}):
    graph = tf.Graph()
    with tf.Session(graph=graph) as sess:
        x = tf.placeholder(tf.float32, (None, 224, 224, 3))

        if attack_type == "FGM":
            attack = FastGradientMethod(model, sess=sess)
            
            ########################################################################
            # NEED TO CROSS CHECK IF IT IS RIGHT
            y_target_vec = np.zeros((len(imgs), 1000))
            y_target_vec[np.arange(len(imgs)), attack_kwargs['y_target']] = 1
            attack_kwargs['y_target'] = y_target_vec
            ########################################################################
            
        elif attack_type == "PGD":
            attack = ProjectedGradientDescent(model, sess=sess)
        
        adv_imgs = attack.generate(x, **attack_kwargs)
        adv_imgs_eval = sess.run(adv_imgs, feed_dict={x: imgs})

    return adv_imgs_eval

# Initialize model and load image

In [13]:
model = InceptionV1Model()

In [61]:
# # Get input images from url
# panda_urls = []
# correct_class_name = 'giant_panda'

# # Panda1
# img_url = ('https://cbsnews1.cbsistatic.com/hub/i/2016/08/26/cdf56aa8-1f2a-4d44-8cac-ab5993ee7d18/gettyimages-594359398.jpg')
# panda_urls.append(img_url)

# # Panda2
# img_url = ('https://media.nature.com/w800/magazine-assets/d41586-019-00301-y/d41586-019-00301-y_16427380.jpg')
# panda_urls.append(img_url)

# # Panda3
# img_url = ('https://nationalzoo.si.edu/sites/default/files/animals/giantpanda-001.jpg')
# panda_urls.append(img_url)

# # Panda4
# img_url = ('http://assets.worldwildlife.org/photos/479/images/story_full_width/giant-panda-shutterstock_86500690.jpg')
# panda_urls.append(img_url)

# # Panda5
# img_url = ('https://cosmos-images1.imgix.net/file/spina/photo/18828/190502_panda_diet_full.jpg')
# panda_urls.append(img_url)

# # Panda6
# img_url = ('https://kpbs.media.clients.ellingtoncms.com/img/photos/2019/04/29/T13_0236_186_t800.jpg')
# panda_urls.append(img_url)

In [10]:
# Get input images from url
img_urls = []
correct_class_name = 'leopard'

# Leopard 1
img_url = 'https://cdn.britannica.com/30/136130-050-3370E37A/Leopard.jpg'
img_urls.append(img_url)

In [60]:
# Get input images from url
img_urls = []
correct_class_name = 'badger'

# Badger 1
img_url = 'https://cdn.britannica.com/01/172401-050-09D09870/European-badger.jpg'
img_urls.append(img_url)

# Badger 2
img_url = 'https://pbs.twimg.com/media/EGLbLOhWoAEvoap.jpg'
img_urls.append(img_url)

# Badger 3
img_url = 'https://static.independent.co.uk/s3fs-public/thumbnails/image/2018/12/18/14/badger-sitting.jpg'
img_urls.append(img_url)

# Badger 4
img_url = 'https://static.wixstatic.com/media/0c3b82_d4a8e1ab9bb6494390c96e1e53fbbb6f~mv2_d_4000_2667_s_4_2.jpeg/v1/fill/w_640,h_718,al_c,q_85,usm_0.66_1.00_0.01/0c3b82_d4a8e1ab9bb6494390c96e1e53fbbb6f~mv2_d_4000_2667_s_4_2.webp'
img_urls.append(img_url)

# Badger 5
img_url = 'https://lh3.googleusercontent.com/uc9FBfPm_1irnZhwUzk-_ctGgCJ9159i08GkYl8ynRlL3sLKJHvQaoeJTVHNsvy0q6jF_m4uU4bP9v8Y-QpGTQ=w1522'
img_urls.append(img_url)

# Badger 6
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRiAOiXbcycg9unmFXa3dLoCzUQ04TVsMZAiw6GMyP6mLsNRCwC&s'
img_urls.append(img_url)

# Badger 7
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSw5XhD6mVGAJuN9jeU0yigw010M8xTqKbPPZPSrYgtvyJLS2ECcg&s'
img_urls.append(img_url)

# Badger 8
img_url = 'https://news.images.itv.com/image/file/1977933/stream_img.jpg'
img_urls.append(img_url)

# Badger 9
img_url = 'https://images.unsplash.com/photo-1542462153-051e01f22e4c?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80'
img_urls.append(img_url)

# Badger 10
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS_vlvSbXELC4PAL9Ps6MQSrQr7FyqrwHrjbxkSJXjaCf6Ymru8&s'
img_urls.append(img_url)

# Badger 11
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcThuZzRbEeQoQIYJfCM35MndBBrkUizkTkZxRXIU3pHNp6raNRR&s'
img_urls.append(img_url)

# Badger 12
img_url = 'https://ptes.org/wp-content/uploads/2014/06/Badger.-Free-to-use-credit-Ben-Andrews-resized.jpg'
img_urls.append(img_url)

# Badger 13
img_url = 'https://www.wildlifetrusts.org/sites/default/files/styles/node_hero_default/public/2017-12/Badger%20-%20Credit%20Mark%20Davison.jpg?h=9a60bbcf&itok=Rfs9CsL_'
img_urls.append(img_url)

# Badger 14
img_url = 'https://i.guim.co.uk/img/media/1dde55e82369d738e02cab5af09b9395aed02b7e/691_605_3972_2383/master/3972.jpg?width=300&quality=85&auto=format&fit=max&s=0cf8e8c18bf616f9df35ed01f20241a9'
img_urls.append(img_url)

# Badger 15
img_url = 'https://fwi-wp-assets-live.s3-eu-west-1.amazonaws.com/sites/1/2019/10/231019-badger-shutterstock_editorial_5307675a.jpg'
img_urls.append(img_url)

# Badger 16
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTLM1gow0NVkAo36hyHHZ7Frdftx2K3dtFISB9o0uIEtDVAZ2vXMw&s'
img_urls.append(img_url)

# Badger 17
img_url = 'https://cdn2-www.mandatory.com/assets/uploads/2017/10/badger2.jpg'
img_urls.append(img_url)

# Badger 18
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT28xTBwNUsq7K-Xa7UnxcpPHj2D7zWxylthJOdlDhBTdr_kmIy&s'
img_urls.append(img_url)

# Badger 18
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRswa9pPToG4Hzb9KyTfC-sgPUbtxIC42n1e2LDETjWK8XSzqrV&s'
img_urls.append(img_url)

# Badger 19
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRwISIIRRhOQpKTK130YuB1hIvF8R12rWdkQpT3v91atEtehZpX&s'
img_urls.append(img_url)
            
# Badger 20
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQQRLjkkPzCMkwTBsUcGNtrb3QX6xXxOzK2KXVEE1faP_9ekquP&s'
img_urls.append(img_url)

# Badger 21
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRG__is2GRsSloqBtD1AQ2dwHzyVV28Ml2g_1XCFytkW-Vped04&s'
img_urls.append(img_url)

# Badger 22
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRTy1BJYkdAEFyry86dAgIaduqI35Ab62IRFECWD6fkUae0qbdW&s'
img_urls.append(img_url)

# Badger 23
img_url = 'https://previews.123rf.com/images/avslt/avslt1601/avslt160100047/50587927-badger-on-the-grass-european-badger-meles-meles.jpg'
img_urls.append(img_url)

# Badger 24
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRQydbpK70eJ6hxtDlRcKnY-RJrdU_sM-FPvowAq6__IJ59ix0O&s'
img_urls.append(img_url)

# Badger 25
img_url = 'https://babycrowyoga.files.wordpress.com/2017/05/badger.jpg?w=898'
img_urls.append(img_url)

# Badger 26
img_url = 'https://www.thecourier.co.uk/wp-content/uploads/sites/12/2019/10/5d9d9998acf5d5.25827003-372x372.jpg'
img_urls.append(img_url)

# Badger 27
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR6OXNN8OJJJFJQXhGqlLd5L5I20GzQAV8SPXgtv4ksvBSXfE85&s'
img_urls.append(img_url)

# Badger 28
img_url = 'https://previews.123rf.com/images/mikelane45/mikelane451312/mikelane45131200859/24424746-badger-meles-meles-single-mammal-uk.jpg'
img_urls.append(img_url)

# Badger 29
img_url = 'https://www.thetimes.co.uk/imageserver/image/methode%2Ftimes%2Fprod%2Fweb%2Fbin%2F485f63be-1f71-11ea-bf0a-65463b7bd0d0.jpg?crop=7360%2C4140%2C0%2C383&resize=685'
img_urls.append(img_url)

# Badger 30
img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSk2Ur6biTm64aOl-S3I4nW5rjoyzoBwIHnhcsb4jvpHi6I-a7O&s'
img_urls.append(img_url)

# Badger 31
img_url = 'https://www.bowgroup.org/sites/bowgroup.uat.pleasetest.co.uk/files/styles/galleryformatter_slide/public/badgerweb.jpg?itok=m63Mk5jr'
img_urls.append(img_url)

# Badger 32
img_url = 'https://babycrowyoga.files.wordpress.com/2017/05/badger.jpg'
img_urls.append(img_url)

# Experiments

### Get class-name mapping

In [9]:
label_df = pd.read_csv('../data/imagenet_tfrecords_labels.txt', sep=' ', names=['syn', 'label', 'name'])
label_dict = {int(label): name for label, name in zip(label_df['label'], label_df['name'])}

### Get prediction on benign images and save them

In [61]:
benign_imgs = []

for i, img_url in enumerate(img_urls):
    # Get prediction
    img = load_image_from_url(img_url)
    predicted_class = get_predictions(model, [img])[0]
    predicted_class_name = label_dict[predicted_class]
    
    # Collect the benign img
    benign_imgs.append(img)
    
    # print out the results
    print(predicted_class_name, predicted_class)
#     plt.imshow(img)
#     plt.show()
    
    # Save the image
    filename = '../data/sample-images/sample-benign/benign-{}-{}.jpg'.format(correct_class_name, i)
    cv2.imwrite(filename, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16
badger 16


### Generate attacked images with different strength and save them

In [62]:
# Target to hen

# Target to armadillo
# target_label = 178

# Target to weasel
target_label = 48 

# Target to impala
# target_label = 57 

# Target to cradle
# target_label = 297 

# Different attack strength
eps_list = [0.5, 1, 1.5, 2, 2.5, 3, 3.5]

for eps in eps_list:
    
    # Generate attacked images
    adv_imgs = attack_images(model, benign_imgs, attack_type="FGM", attack_kwargs={'eps': eps, 'y_target': target_label})
    
    # Save the attacked images
    for i, adv_img in enumerate(adv_imgs):
        
        # Get the predicted class
        predicted_class = get_predictions(model, [adv_img])[0]
        predicted_class_name = label_dict[predicted_class]

        # Display the attacked image
        attacked_img = np.array(adv_img, dtype=np.uint8)
        print('{}-th image, predicted as {}, eps={}'.format(i, predicted_class_name, eps))
#         plt.imshow(attacked_img)
#         plt.show()

        # Save the attacked image if it the targeted attack succeed
        if target_label == predicted_class:
            dirname = '../data/sample-images/attacked-{}'.format(eps)
            i = int(len(os.listdir(dirname)) / 2)
            filename = '{}/{}'.format(dirname, i)
            np.save(filename, adv_img)
            cv2.imwrite(filename + '.jpg', cv2.cvtColor(attacked_img, cv2.COLOR_RGB2BGR))

0-th image, predicted as badger, eps=0.5
1-th image, predicted as weasel, eps=0.5
2-th image, predicted as weasel, eps=0.5
3-th image, predicted as weasel, eps=0.5
4-th image, predicted as badger, eps=0.5
5-th image, predicted as weasel, eps=0.5
6-th image, predicted as weasel, eps=0.5
7-th image, predicted as badger, eps=0.5
8-th image, predicted as polecat, eps=0.5
9-th image, predicted as badger, eps=0.5
10-th image, predicted as badger, eps=0.5
11-th image, predicted as badger, eps=0.5
12-th image, predicted as badger, eps=0.5
13-th image, predicted as badger, eps=0.5
14-th image, predicted as mongoose, eps=0.5
15-th image, predicted as Arctic_fox, eps=0.5
16-th image, predicted as polecat, eps=0.5
17-th image, predicted as badger, eps=0.5
18-th image, predicted as coral_fungus, eps=0.5
19-th image, predicted as weasel, eps=0.5
20-th image, predicted as badger, eps=0.5
21-th image, predicted as hare, eps=0.5
22-th image, predicted as coral_fungus, eps=0.5
23-th image, predicted as 